In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']
bnt_funding_limit: Decimal = Decimal('40000')
bnt_min_liquidity: Decimal = Decimal('10000')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[1.0],
                            'tkn':[3.0],
                            # 'link':[15.00],
                            # 'eth':[2500.00],
                            # 'wbtc':[40000.00],
                            })

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
# v3 = BancorV3(whitelisted_tokens=whitelisted_tokens,
#               network_fee=network_fee,
#               trading_fee=trading_fee,
#               cooldown_time=cooldown_time,
#               withdrawal_fee=withdrawal_fee,
#               bnt_min_liquidity=bnt_min_liquidity,
#               bnt_funding_limit=bnt_funding_limit,
#               price_feeds=price_feeds)

# v3.create_user('Alice')
# v3.create_user('Bob')
# v3.create_user('Charlie')
#
# # The actual deposits.
#
# v3.stake(tkn_amt=100, tkn_name='eth', user_name='Alice')
#
# v3.stake(tkn_amt=10000, tkn_name='link', user_name='Charlie')
#
# v3.stake(tkn_amt=100, tkn_name='wbtc', user_name='Bob')
#
# v3.stake(tkn_amt=1, tkn_name='eth', user_name='Alice')
#
# v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie')
#
# v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob')
#
# v3.dao_msig_init_pools(whitelisted_tokens)
#
# v3.create_user('Trader')
#
# v3.trade(swap_amount=2000, source_token='bnt',  target_token='link', user_name='Trader', timestep=0)
#
# v3.trade(swap_amount=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestep=0)
#
# v3.trade(swap_amount=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestep=0)

def read_url(url):
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

In [2]:
# <p align="center">
# <img width="100%" src="https://d9hhrg4mnvzow.cloudfront.net/try.bancor.network/5edb37d6-open-graph-bancor3_1000000000000000000028.png" alt="bancor3" />
# </p>
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# # Simulation Example

In [3]:
v3 = BancorV3(active_users=['dayTrader', 'tknProvider', 'bntProvider'],
              whitelisted_tokens=['tkn','bnt'])

v3.list_tokens, v3.list_users

(['tkn', 'bnt'], ['dayTrader', 'tknProvider', 'bntProvider'])

In [4]:
v3.upload_json_simulation(path='/Users/mikewcasale/Local/projects/bancor/simulator-v3/bancor3_simulator/tests/data/scenarios_2.json')
v3.list_tokens, v3.list_users

(['tkn', 'bnt'], ['dayTrader', 'tknProvider', 'bntProvider'])

In [5]:
v3.describe()

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol Wallet
0,,bnt=0.000000,bnt=0.000000,bnbnt=0.000000,bnt=0.000000,tkn=0.000000,bnbnt=0.000000
1,bnt=0.000000 tkn=0.000000,tkn=0.000000,tkn=0.000000,bntkn=0.000000,,bnt=0.000000,
2,,,,,,,


In [6]:
timestep = 0
simulation = []
validation = []
# QDECIMALS = Decimal(10) ** -18
indx = 0
operations_count = len(v3.json_data['operations'])

In [7]:
from bancor3_simulator.core.constants import QDECIMALS

for transaction in v3.json_data['operations']:

    # setup
    indx += 1
    transaction_type = transaction['type']
    user_name = transaction['userId']
    mined = transaction['mined']
    amt = transaction['amount']
    if type(amt) == dict:
        v3.price_feeds = pd.DataFrame({'INDX':[i for i in range(operations_count)],
                            'tkn':[Decimal(amt['bntVirtualBalance']) for i in range(operations_count)],
                            'bnt':[Decimal(amt['baseTokenVirtualBalance']) for i in range(operations_count)],
        })
        amt = Decimal(0)
    else:
        amt = Decimal(amt)

    tkn_name = 'bnt' if 'BNT' in transaction_type else 'tkn'
    target_tkn = 'tkn' if tkn_name == 'bnt' else 'bnt'
    if mined:
        v3.global_state.timestep += 1
        timestep = v3.global_state.timestep
    if 'enable' in transaction_type:
        v3.dao_msig_init_pools(['bnt','tkn'])

    # action
    elif 'deposit' in transaction_type:
        v3.stake(tkn_name=tkn_name, tkn_amt=amt, user_name=user_name, timestep=timestep)
    elif 'trade' in transaction_type:
        v3.trade(source_token=tkn_name, target_token=target_tkn, swap_amount=amt, user_name=user_name, timestep=timestep)
    elif 'withdraw' in transaction_type:
        id_number = v3.begin_cooldown(tkn_name=tkn_name,
                                      withdraw_value=amt,
                                      user_name=user_name,
                                      timestep=timestep)
        v3.withdraw(user_name=user_name, id_number=id_number, timestep=timestep)

    # logging
    expected = transaction['expected']
    collect = {
          "indx": [indx],
          "action": [transaction_type],
          "amount": [amt],
          "tkn_dayTrader": [Decimal(expected['tknBalances']['dayTrader']).quantize(QDECIMALS)],
          "tkn_tknProvider": [Decimal(expected['tknBalances']['tknProvider']).quantize(QDECIMALS)],
          "tkn_bntProvider": [Decimal(expected['tknBalances']['bntProvider']).quantize(QDECIMALS)],
          "tkn_masterVault": [Decimal(expected['tknBalances']['masterVault']).quantize(QDECIMALS)],
          "tkn_epVault": [Decimal(expected['tknBalances']['epVault']).quantize(QDECIMALS)],
          "bnt_dayTrader": [Decimal(expected['bntBalances']['dayTrader']).quantize(QDECIMALS)],
          "bnt_tknProvider": [Decimal(expected['bntBalances']['tknProvider']).quantize(QDECIMALS)],
          "bnt_bntProvider": [Decimal(expected['bntBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnt_masterVault": [Decimal(expected['bntBalances']['masterVault']).quantize(QDECIMALS)],
          "bntkn_dayTrader": [Decimal(expected['bntknBalances']['dayTrader']).quantize(QDECIMALS)],
          "bntkn_tknProvider": [Decimal(expected['bntknBalances']['tknProvider']).quantize(QDECIMALS)],
          "bntkn_bntProvider": [Decimal(expected['bntknBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnbnt_dayTrader": [Decimal(expected['bnbntBalances']['dayTrader']).quantize(QDECIMALS)],
          "bnbnt_tknProvider": [Decimal(expected['bnbntBalances']['tknProvider']).quantize(QDECIMALS)],
          "bnbnt_bntProvider": [Decimal(expected['bnbntBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnbnt_bntPool": [Decimal(expected['bnbntBalances']['bntPool']).quantize(QDECIMALS)],
          "bntCurrentPoolFunding": [Decimal(expected['bntCurrentPoolFunding']).quantize(QDECIMALS)],
          "tknStakedBalance": [Decimal(expected['tknStakedBalance']).quantize(QDECIMALS)],
          "bntStakedBalance": [Decimal(expected['bntStakedBalance']).quantize(QDECIMALS)],
          "tknTradingLiquidity": [Decimal(expected['tknTradingLiquidity']).quantize(QDECIMALS)],
          "bntTradingLiquidity": [Decimal(expected['bntTradingLiquidity']).quantize(QDECIMALS)],
    }
    validation.append(pd.DataFrame(collect))

    collect = {
          "indx": [indx],
          "action": [transaction_type],
          "amount": [amt],
          "tkn_dayTrader": [v3.global_state.users['dayTrader'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
          "tkn_tknProvider": [v3.global_state.users['tknProvider'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
          "tkn_bntProvider": [v3.global_state.users['bntProvider'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
          "tkn_masterVault": [v3.global_state.tokens['tkn'].vault_tkn.quantize(QDECIMALS)],
          "tkn_epVault": [v3.global_state.tokens['tkn'].external_protection_vault.quantize(QDECIMALS)],
          "bnt_dayTrader": [v3.global_state.users['dayTrader'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
          "bnt_tknProvider": [v3.global_state.users['tknProvider'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
          "bnt_bntProvider": [v3.global_state.users['bntProvider'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
          "bnt_masterVault": [v3.global_state.vault_bnt.quantize(QDECIMALS)],
          "bntkn_dayTrader": [v3.global_state.users['dayTrader'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
          "bntkn_tknProvider": [v3.global_state.users['tknProvider'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
          "bntkn_bntProvider": [v3.global_state.users['bntProvider'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_dayTrader": [v3.global_state.users['dayTrader'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_tknProvider": [v3.global_state.users['tknProvider'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_bntProvider": [v3.global_state.users['bntProvider'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_bntPool": [v3.global_state.protocol_wallet_bnbnt.quantize(QDECIMALS)],
          "bntCurrentPoolFunding": [v3.global_state.tokens['tkn'].bnt_funding_amount.quantize(QDECIMALS)],
          "tknStakedBalance": [v3.global_state.tokens['tkn'].staked_tkn.quantize(QDECIMALS)],
          "bntStakedBalance": [v3.global_state.staked_bnt.quantize(QDECIMALS)],
          "tknTradingLiquidity": [v3.global_state.tokens['tkn'].tkn_trading_liquidity.quantize(QDECIMALS)],
          "bntTradingLiquidity": [v3.global_state.tokens['tkn'].bnt_trading_liquidity.quantize(QDECIMALS)],
    }
    simulation.append(pd.DataFrame(collect))


case1
trade2  3 0.0025 0.20 2000
trade1  4 0.0025 0.20 5332
case1
trade1  7 0.0025 0.20 213
trade2  8 0.0025 0.20 613
trade1  10 0.0025 0.20 191
case2
trade1  16 0.0025 0.20 210
trade2  18 0.0025 0.20 803
trade2  21 0.0025 0.20 583
trade2  22 0.0025 0.20 142
trade2  28 0.0025 0.20 848
trade2  28 0.0025 0.20 886
trade1  31 0.0025 0.20 42
trade2  33 0.0025 0.20 746
trade2  35 0.0025 0.20 360
trade2  37 0.0025 0.20 604
trade2  38 0.0025 0.20 121
trade1  47 0.0025 0.20 158
trade1  55 0.0025 0.20 25
trade1  66 0.0025 0.20 185
trade2  71 0.0025 0.20 424
trade2  73 0.0025 0.20 582
trade2  77 0.0025 0.20 161
trade2  78 0.0025 0.20 541
trade1  91 0.0025 0.20 196
trade1  97 0.0025 0.20 316
trade2  98 0.0025 0.20 210
trade1  101 0.0025 0.20 139
trade2  102 0.0025 0.20 425
trade1  108 0.0025 0.20 38
trade1  109 0.0025 0.20 126
trade2  110 0.0025 0.20 149
trade1  114 0.0025 0.20 130
trade2  115 0.0025 0.20 409
trade2  119 0.0025 0.20 359
trade1  124 0.0025 0.20 248
trade1  125 0.0025 0.20 42
trade1

In [8]:
v3.global_state.trading_fee, v3.trading_fee, v3.global_state.tokens['tkn'].trading_fee

(Decimal('0.0025'), Decimal('0.0025'), Decimal('0.0025'))

In [9]:
v3.global_state.bnt_min_liquidity, v3.global_state.bnt_funding_limit

(Decimal('1000000'), Decimal('10000000'))

In [10]:
simulation, validation = pd.concat(simulation), pd.concat(validation)

In [11]:
simulation = simulation.drop('indx', axis=1).reset_index().drop('index', axis=1)
simulation

,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,0E-18,0E-18,30000000.000000000000000000,0E-18,0E-18,0E-18
1,enableTrading,0,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,depositTKN,1000000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,depositBNT,100000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,tradeTKN,2000,8000.000000000000000000,69000000.000000000000000000,0E-18,31002000.000000000000000000,0E-18,10664.889185135810698217,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,depositTKN,36883,17849.329770407740647656,62102442.000000000000000000,0E-18,37889708.670229592259352344,0E-18,7318.712463663813911981,0E-18,6783426.000000000000000000,...,0E-18,0E-18,0E-18,1735099.055019285427254587,6783422.156707742187958060,10000019.117811980227248122,37897636.494896782772412326,10000019.117811980227248122,30023259.277360642248949420,10002667.784755823310177166
395,depositBNT,3304,17849.329770407740647656,62102442.000000000000000000,0E-18,37889708.670229592259352344,0E-18,7318.712463663813911981,0E-18,6780122.000000000000000000,...,0E-18,0E-18,0E-18,1738403.047432951090417753,6780118.156707742187958060,10000019.117811980227248122,37897636.494896782772412326,10000019.117811980227248122,30023259.277360642248949420,10002667.784755823310177166
396,withdrawBNT,69534,17849.329770407740647656,62102442.000000000000000000,0E-18,37889708.670229592259352344,0E-18,7318.712463663813911981,0E-18,6780122.000000000000000000,...,0E-18,0E-18,0E-18,1668869.207090388069114012,6780118.156707742187958060,10000019.117811980227248122,37897636.494896782772412326,10000019.117811980227248122,30023259.277360642248949420,10002667.784755823310177166
397,tradeBNT,36,17957.114151895984338224,62102442.000000000000000000,0E-18,37889600.885848104015661776,0E-18,7282.712463663813911981,0E-18,6780122.000000000000000000,...,0E-18,0E-18,0E-18,1668869.207090388069114012,6780118.156707742187958060,10000019.117811980227248122,37897636.711005818337813209,10000019.117811980227248122,30023151.492979154005258852,10002703.766755758689417176


In [12]:
validation = validation.drop('indx', axis=1).reset_index().drop('index', axis=1)
validation

,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,0E-18,0E-18,30000000.000000000000000000,0E-18,0E-18,0E-18
1,enableTrading,0,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,depositTKN,1000000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,depositBNT,100000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,tradeTKN,2000,8000.000000000000000000,69000000.000000000000000000,0E-18,31002000.000000000000000000,0E-18,10664.889185135810698217,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,depositTKN,36883,17843.180340779883503492,64004314.664415812819109221,0E-18,35987842.155243407297387287,0E-18,7320.922988706987401096,0E-18,8261197.623349625732944047,...,0E-18,0E-18,0E-18,1735096.742662854896616032,8262253.649874946739798897,9997373.358752083284130504,35990997.219994855880358585,9997373.358752083284130504,30002722.540170501638457541,10000000.000000000000000000
395,depositBNT,3304,17843.180340779883503492,64004314.664415812819109221,0E-18,35987842.155243407297387287,0E-18,7320.922988706987401096,0E-18,8257893.623349625732944047,...,0E-18,0E-18,0E-18,1738400.735072824069155190,8258949.657464977567259739,9997373.358752083284130504,35990997.219994855880358585,9997373.358752083284130504,30002722.540170501638457541,10000000.000000000000000000
396,withdrawBNT,69534,17843.180340779883503492,64004314.664415812819109221,0E-18,35987842.155243407297387287,0E-18,7320.922988706987401096,0E-18,8327253.947685884787087112,...,0E-18,0E-18,0E-18,1668866.735072824069155190,8328483.657464977567259739,9997373.358752083284130504,35990997.219994855880358585,9997373.358752083284130504,30002722.540170501638457541,10000000.000000000000000000
397,tradeBNT,36,17950.919729559836167046,64004314.664415812819109221,0E-18,35987734.415854627344723733,0E-18,7284.922988706987401096,0E-18,8327253.947685884787087112,...,0E-18,0E-18,0E-18,1668866.735072824069155190,8328483.657464977567259739,9997373.358752083284130504,35990997.436013680501817560,9997373.358752083284130504,30002614.800781721685793987,10000035.981999935362000582


In [13]:
result_df = (1 - simulation.drop('action', axis=1).replace(0,Decimal(0.0000000000000000001)) / validation.drop('action', axis=1).replace(0,Decimal(0.0000000000000000001))) * 100
result_df

,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,bnt_masterVault,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,0,-0.0344637531561728700123130868669120095120571...,2.97147571126667855447104963287414241595398149...,0,-5.2847472954392852827034983202625549623520755...,0,0.03019462227076246550098502996151354051636253...,0,17.8881040101597424065003817785701926819477162...,-0.0266778050989036959874511547432680575062618...,...,0,0,0,-0.0001332695966555653014918836024714682139787...,17.8986455249965291443416771366246198796277476...,-0.0264645418847015888538894709068269564166039...,-5.2975450034007126744091191785832445151625598...,-0.0264645418847015888538894709068269564166039...,-0.0684495787428759884755615639682029138856437...,-0.0266778475582331017716600
395,0,-0.0344637531561728700123130868669120095120571...,2.97147571126667855447104963287414241595398149...,0,-5.2847472954392852827034983202625549623520755...,0,0.03019462227076246550098502996151354051636253...,0,17.8952610768822360166862944367843482243809836...,-0.0266778050989036959874511547432680575062618...,...,0,0,0,-0.0001330165180196149952691759422200306458653...,17.9058059691715275254882946659543100870729381...,-0.0264645418847015888538894709068269564166039...,-5.2975450034007126744091191785832445151625598...,-0.0264645418847015888538894709068269564166039...,-0.0684495787428759884755615639682029138856437...,-0.0266778475582331017716600
396,0,-0.0344637531561728700123130868669120095120571...,2.97147571126667855447104963287414241595398149...,0,-5.2847472954392852827034983202625549623520755...,0,0.03019462227076246550098502996151354051636253...,0,18.5791373411378601072489481352240255850858527...,-0.0266778050989036959874511547432680575062618...,...,0,0,0,-0.0001481255220712448192797341871606324658083...,18.5912053674909485970509309406542386228855103...,-0.0264645418847015888538894709068269564166039...,-5.2975450034007126744091191785832445151625598...,-0.0264645418847015888538894709068269564166039...,-0.0684495787428759884755615639682029138856437...,-0.0266778475582331017716600
397,0,-0.0345075485238107124344033077417973852324460...,2.97147571126667855447104963287414241595398149...,0,-5.2847629917919956432087850726305941588700157...,0,0.03034383543381614704138332978760821048679014...,0,18.5791373411378601072489481352240255850858527...,-0.0266777090592640456777391416099062166871191...,...,0,0,0,-0.0001481255220712448192797341871606324658083...,18.5912053674909485970509309406542386228855103...,-0.0264645418847015888538894709068269564166039...,-5.2975449718553699081191762160121793596955803...,-0.0264645418847015888538894709068269564166039...,-0.0684496745826874846748247994664351719366407...,-0.0266777515663477605730769278681117757751894...


In [14]:
# comparison = simulation.compare(validation)
# comparison

In [15]:
# comparison = comparison.reset_index()
# comparison

In [16]:
# import seaborn as sns
# import matplotlib.pyplot as plt
#
# for col in result_df.columns:
#     try:
#         fig = plt.figure()
#         fig.set_size_inches(6,6)
#         print('\n\n')
#         print(f"Compare {col}:")
#         sns.lineplot(data = result_df, x = 'index', y = col, color='r')
#         plt.show()
#     except:
#         pass